In [1]:
!pip install qdrant-client

In [10]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from pathlib import Path
from dotenv import load_dotenv,find_dotenv

from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS,Qdrant

In [11]:
load_dotenv(find_dotenv("../.env"))

True

In [12]:
filePath=Path("data").joinpath("letters").joinpath("state_of_the_union.txt")

In [13]:
rawDocument=TextLoader(file_path=filePath,encoding="utf-8").load()
splittedDocuments=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=10).split_documents(documents=rawDocument)

In [14]:
db=FAISS.from_documents(documents=splittedDocuments,embedding=OpenAIEmbeddings())

<h3>Similarity Search</h3>

In [15]:
query="What did the president say about Ketanji Brown Jackson?"

In [16]:
docs=db.similarity_search(query=query,k=2)
docs

[Document(id='4fe7ea92-e74a-4f5d-b898-cebc26a39308', metadata={'source': 'data\\letters\\state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(id='adde7165-11f2-4565-a288-e4d08ed5fb7f', metadata={'source': '

<h3>Similarity Search by Vector</h3>

In [18]:
embedding_vector=OpenAIEmbeddings().embed_query(text=query)

In [19]:
docs=db.similarity_search_by_vector(embedding=embedding_vector,k=2)

In [20]:
docs

[Document(id='4fe7ea92-e74a-4f5d-b898-cebc26a39308', metadata={'source': 'data\\letters\\state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(id='adde7165-11f2-4565-a288-e4d08ed5fb7f', metadata={'source': '

<h3>Use QDRANT from Async Vectorstore</h3>

In [26]:
# start the docker dontainer for qdrant
db = await Qdrant.afrom_documents(documents=splittedDocuments, embedding=OpenAIEmbeddings(), url="http://localhost:6333")

In [27]:
docs=db.similarity_search(query=query,k=2)
docs

[Document(metadata={'source': 'data\\letters\\state_of_the_union.txt', '_id': '21ee8f51-3feb-4712-a31b-082c32402565', '_collection_name': '0f3b5b2cf0da4df789f4d597b8ea6d19'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(meta